## Loading Data & Importing Libraries

In [1]:
# Load libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import os
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import csr_matrix
import implicit

In [2]:
# import datasets
artists = pd.read_csv(os.path.join('..','data','artists.dat'), delimiter='\t')
tags = pd.read_csv(os.path.join('..','data','tags.dat'), delimiter='\t',encoding='ISO-8859-1')
user_artists = pd.read_csv(os.path.join('..','data','user_artists.dat'), delimiter='\t')
user_friends = pd.read_csv(os.path.join('..','data','user_friends.dat'), delimiter='\t')
user_taggedartists_timestamps = pd.read_csv(os.path.join('..','data','user_taggedartists-timestamps.dat'), delimiter='\t')
user_taggedartists = pd.read_csv(os.path.join('..','data','user_taggedartists.dat'), delimiter='\t')

## Data Cleaning

In [3]:
# Drop irrelevant columns from the Artists dataset
artists_cleaned = artists.drop(columns=['url', 'pictureURL']).drop_duplicates(keep='first') 

# Drop the irrelevant columns in the Tags dataset
tags_cleaned = tags.drop_duplicates(keep='first') 

# For the User-Artists dataset, we can filter out rows with a weight of 0, as they show no meaningful interaction
# user_artists_cleaned = user_artists[user_artists['weight'] > 0]
user_artists_cleaned = user_artists.drop_duplicates(keep='first') 

# Drop duplicates from the User-Tagged Artists Timestamps dataset
user_taggedartists_timestamps_cleaned = user_taggedartists_timestamps.drop_duplicates(keep='first') 

# Convert timestamps from ms to datetime format
user_taggedartists_timestamps_cleaned['timestamp'] = pd.to_datetime(user_taggedartists_timestamps_cleaned['timestamp'], unit='ms')

# Drop duplicates from the User-Friends dataset
user_friends_cleaned = user_friends.drop_duplicates(keep='first') 

# # Output cleaned datasets for inspection
print("Cleaned Artists dataset:", artists_cleaned.info(), artists_cleaned.head())
print("Cleaned Tags dataset:", tags_cleaned.info(), tags_cleaned.head())
print("Cleaned User-Artists dataset:", user_artists_cleaned.info(), user_artists_cleaned.head())
print("Cleaned User-Tagged Artists Timestamps dataset:", user_taggedartists_timestamps_cleaned.info(), user_taggedartists_timestamps_cleaned.head())
print("Cleaned User-Friends dataset:", user_friends_cleaned.info(), user_friends_cleaned.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17632 entries, 0 to 17631
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   id      17632 non-null  int64 
 1   name    17632 non-null  object
dtypes: int64(1), object(1)
memory usage: 275.6+ KB
Cleaned Artists dataset: None    id               name
0   1       MALICE MIZER
1   2    Diary of Dreams
2   3  Carpathian Forest
3   4       Moi dix Mois
4   5        Bella Morte
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 11946 entries, 0 to 11945
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   tagID     11946 non-null  int64 
 1   tagValue  11946 non-null  object
dtypes: int64(1), object(1)
memory usage: 186.8+ KB
Cleaned Tags dataset: None    tagID           tagValue
0      1              metal
1      2  alternative metal
2      3          goth rock
3      4        black metal
4      5        death metal
<class 'p

---

# Collaborative Filtering

We now implement collaborative filtering using some of the different techniques that we have described. We will use some memory-based and some model-based methods.
First, we create a dictionary which will allow us to map artistID recommendations to the corresponding names of the artists.

In [4]:
# Create a dictionary to map artistID to artistName
artist_id_to_name = dict(zip(artists['id'], artists['name']))

We initialise the user-artist matrix which has values corresponding to the listening counts.

In [5]:
# Create a user-artist interaction matrix using the user_artists_cleaned dataset
user_artist_matrix = user_artists_cleaned.pivot(index='userID', columns='artistID', values='weight')

# Fill NaN values with 0s (assuming binary or implicit feedback, i.e., 1 for interaction, 0 for no interaction)
user_artist_matrix = user_artist_matrix.fillna(0)

print(user_artist_matrix)

artistID  1      2      3      4      5      6      7      8      9      \
userID                                                                    
2           0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
3           0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
4           0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
5           0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
6           0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
...         ...    ...    ...    ...    ...    ...    ...    ...    ...   
2095        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2096        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2097        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2099        0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0    0.0   
2100        0.0    0.0  408.0    0.0    0.0  404.0    0.0    0.0    0.0   

artistID  10     ...  18

## Memory-based Collaborative Filtering

### User-Based Implementation
For the user-based implementation, we must compute the similarity matrix using the cosine similarity between users.

In [6]:
# Compute the cosine similarity between users
user_similarity = cosine_similarity(user_artist_matrix)

# Convert the similarity matrix into a DataFrame for easy inspection
user_similarity_df = pd.DataFrame(user_similarity, index=user_artist_matrix.index, columns=user_artist_matrix.index)

# Display a portion of the user similarity matrix
print(user_similarity_df.head())

userID      2     3         4         5         6         7         8     \
userID                                                                     
2       1.000000   0.0  0.144786  0.028692  0.007016  0.030219  0.008964   
3       0.000000   1.0  0.000000  0.000000  0.000000  0.000000  0.000000   
4       0.144786   0.0  1.000000  0.081193  0.006609  0.000000  0.000000   
5       0.028692   0.0  0.081193  1.000000  0.000000  0.000000  0.000000   
6       0.007016   0.0  0.006609  0.000000  1.000000  0.012713  0.018881   

userID  9         10        11    ...      2090      2091      2092      2093  \
userID                            ...                                           
2        0.0  0.000000  0.021267  ...  0.000000  0.043405  0.000000  0.004625   
3        0.0  0.000000  0.000000  ...  0.000000  0.000000  0.000000  0.000000   
4        0.0  0.009072  0.013407  ...  0.000000  0.000000  0.003776  0.006178   
5        0.0  0.169078  0.004639  ...  0.010993  0.000000  0.2

We create a function which gives user-based recommendations. This function uses the user-artist matrix and the similarity matrix to give the top-$N$ recommendations for the given user. It then maps the corresponding ID recommendations of artists to the artist names using the previously defined dictionary.

The implementation is shown for user 2.

In [7]:
# Function to get user-based recommendations
def get_user_based_recommendations(user_id, user_similarity_df, user_artist_matrix, artist_id_to_name, top_n=10):    
    # Get the most similar users (excluding the user itself)
    similar_users = user_similarity_df[user_id].sort_values(ascending=False).index[1:]

    recommendations = {}
    for similar_user in similar_users:
        # Get the artists this similar user has interacted with (non-zero values)
        interacted_artists = user_artist_matrix.loc[similar_user][user_artist_matrix.loc[similar_user] > 0].index.tolist()

        for artist in interacted_artists:
            # Only consider artists the target user has not interacted with
            if artist not in user_artist_matrix.loc[user_id][user_artist_matrix.loc[user_id] > 0].index.tolist():
                # Add the artist to recommendations with a score (using the scaled similarity as a weight)
                if artist not in recommendations:
                    recommendations[artist] = user_similarity_df[user_id][similar_user]
                else:
                    # Add the weight of similarity to the current score
                    recommendations[artist] += user_similarity_df[user_id][similar_user]

    # Sort recommendations by score (highest first)
    sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)

    # Convert artist IDs to names and prepare the final list with IDs, names, and scores
    recommended_artists = [(artist, artist_id_to_name.get(artist, "Unknown"), score) for artist, score in sorted_recommendations[:top_n]]

    return recommended_artists

In [8]:
# Example: Get top 5 user-based recommendations for user with userID=2
user_id = 2
user_based_recommendations = get_user_based_recommendations(user_id, user_similarity_df, user_artist_matrix, artist_id_to_name, top_n=5)

# Display user-based recommendations
print("Top User-Based Recommendations for User 2:")
for artist_id, artist_name, score in user_based_recommendations:
    print(f"Artist ID: {artist_id}, Artist: {artist_name}, Similarity Score: {score:.2f}")

Top User-Based Recommendations for User 2:
Artist ID: 289, Artist: Britney Spears, Similarity Score: 20.67
Artist ID: 288, Artist: Rihanna, Similarity Score: 20.10
Artist ID: 295, Artist: Beyoncé, Similarity Score: 16.92
Artist ID: 292, Artist: Christina Aguilera, Similarity Score: 16.73
Artist ID: 300, Artist: Katy Perry, Similarity Score: 15.50


We can see that the output is the top 5 recommendations for user 2 with their similarity scores. The output has recommended 5 female pop singers which is interesting. We will now give recommendations for user 3.

In [9]:
# Example: Get top 5 user-based recommendations for user with userID=3
user_id = 3
user_based_recommendations = get_user_based_recommendations(user_id, user_similarity_df, user_artist_matrix, artist_id_to_name, top_n=5)

# Display user-based recommendations
print("Top User-Based Recommendations for User 3:")
for artist_id, artist_name, score in user_based_recommendations:
    print(f"Artist ID: {artist_id}, Artist: {artist_name}, Similarity Score: {score:.2f}")

Top User-Based Recommendations for User 3:
Artist ID: 757, Artist: Crystal Castles, Similarity Score: 0.25
Artist ID: 603, Artist: Aphex Twin, Similarity Score: 0.25
Artist ID: 1222, Artist: Venetian Snares, Similarity Score: 0.23
Artist ID: 2174, Artist: edIT, Similarity Score: 0.23
Artist ID: 154, Artist: Radiohead, Similarity Score: 0.21


The artists recommended for user 3 have much lower similarity scores than those recommended to user 2. In this case, the top 4 recommendations all could be categorised as electronic artists. However, the fifth recommendation 'Radiohead' is not very similar to the others.

### Item-based Implementation
For the item-based implementation, we must compute the similarity matrix using the cosine similarity between artists.

In [10]:
# Compute the cosine similarity between artists (transpose the matrix to compare artists)
artist_similarity = cosine_similarity(user_artist_matrix.T)  # Transpose to compare artists (columns)

# Convert the similarity matrix into a DataFrame for easy inspection
artist_similarity_df = pd.DataFrame(artist_similarity, index=user_artist_matrix.columns, columns=user_artist_matrix.columns)

# Display a portion of the artist similarity matrix
print(artist_similarity_df.head())

artistID  1        2      3      4        5         6         7         8      \
artistID                                                                        
1           1.0  0.00000    0.0    0.0  0.00000  0.000000  0.008784  0.032075   
2           0.0  1.00000    0.0    0.0  0.20774  0.000000  0.010696  0.000000   
3           0.0  0.00000    1.0    0.0  0.00000  0.205607  0.000000  0.000000   
4           0.0  0.00000    0.0    1.0  0.00000  0.000000  0.019742  0.049547   
5           0.0  0.20774    0.0    0.0  1.00000  0.000000  0.042728  0.000000   

artistID     9         10     ...  18736  18737  18738  18739  18740  18741  \
artistID                      ...                                             
1         0.000000  0.000000  ...    0.0    0.0    0.0    0.0    0.0    0.0   
2         0.102094  0.387653  ...    0.0    0.0    0.0    0.0    0.0    0.0   
3         0.000000  0.000000  ...    0.0    0.0    0.0    0.0    0.0    0.0   
4         0.000000  0.000000  ...    

We create a function which gives item-based recommendations. This function uses the user-artist matrix and the similarity matrix to give the top-$N$ recommendations for the given user. It then maps the corresponding ID recommendations of artists to the artist names using the previously defined dictionary.

The implementation is shown for user 2.

In [11]:
# Function to get item-based recommendations
def get_item_based_recommendations(user_id, user_artist_matrix, artist_similarity_df, artist_id_to_name, top_n=10):
    # Get the artists the user has interacted with (non-zero values)
    interacted_artists = user_artist_matrix.loc[user_id][user_artist_matrix.loc[user_id] > 0].index.tolist()
    
    recommendations = {}
    for artist in interacted_artists:
        # Get the most similar artists to the ones the user interacted with
        similar_artists = artist_similarity_df[artist].sort_values(ascending=False).index[1:]  # Exclude the artist itself

        for similar_artist in similar_artists:
            # Add the similar artist to recommendations with a score (using the similarity as a weight)
            if similar_artist not in recommendations:
                recommendations[similar_artist] = artist_similarity_df[artist][similar_artist]
            else:
                recommendations[similar_artist] += artist_similarity_df[artist][similar_artist]

    # Sort recommendations by score (highest first)
    sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)

    # Convert artist IDs to names using artist_id_to_name
    recommended_artists = [(artist_id, artist_id_to_name.get(artist_id, "Unknown"), score) for artist_id, score in sorted_recommendations[:top_n]]

    return recommended_artists

In [12]:
# Example: Get top 5 item-based recommendations for user with userID=2
user_id = 2
item_based_recommendations = get_item_based_recommendations(user_id, user_artist_matrix, artist_similarity_df, artist_id_to_name, top_n=5)

# Display item-based recommendations
print("\nTop Item-Based Recommendations for User 2:")
for artist_id, artist_name, score in item_based_recommendations:
    print(f"Artist ID: {artist_id}, Artist: {artist_name}, Similarity Score: {score:.2f}")



Top Item-Based Recommendations for User 2:
Artist ID: 74, Artist: Basia, Similarity Score: 24.97
Artist ID: 92, Artist: Vitamin Z, Similarity Score: 24.97
Artist ID: 79, Artist: Fiction Factory, Similarity Score: 24.97
Artist ID: 87, Artist: Deacon Blue, Similarity Score: 24.97
Artist ID: 60, Artist: Matt Bianco, Similarity Score: 23.97


We can see that the item-based method gives different recommendations to the user-based method. The similarity scores for the top 5 recommendations are higher for the item-based model.

In [13]:
# Example: Get top 5 item-based recommendations for user with userID=3
user_id = 3
item_based_recommendations = get_item_based_recommendations(user_id, user_artist_matrix, artist_similarity_df, artist_id_to_name, top_n=5)

# Display item-based recommendations
print("\nTop Item-Based Recommendations for User 3:")
for artist_id, artist_name, score in item_based_recommendations:
    print(f"Artist ID: {artist_id}, Artist: {artist_name}, Similarity Score: {score:.2f}")


Top Item-Based Recommendations for User 3:
Artist ID: 134, Artist: Big Brotherz, Similarity Score: 41.77
Artist ID: 131, Artist: Part Timer, Similarity Score: 41.77
Artist ID: 130, Artist: Philippe Lamy, Similarity Score: 41.77
Artist ID: 129, Artist: Aless, Similarity Score: 41.77
Artist ID: 128, Artist: strom noir, Similarity Score: 41.77


Again, we can see that the item-based method gives different recommendations to the user-based method. The similarity scores for the top 5 recommendations are much higher for the item-based model for user 3.

---

## Model-based Methods

### Singular Value Decomposition
We compute the SVD of the user-artist matrix using `scikit-learn` and use this to give recommendations. 

In [14]:
def get_svd_recommendations(user_id, user_artist_matrix, artist_id_to_name, top_n=10, n_components=50):
    # Apply SVD to the user-artist matrix
    svd = TruncatedSVD(n_components=n_components, random_state=42)
    svd_matrix = svd.fit_transform(user_artist_matrix)
    svd_components = svd.components_

    # Reconstruct the user-artist interaction matrix
    reconstructed_matrix = np.dot(svd_matrix, svd_components)
    
    recommendations = {}
        
    # Get the user's interaction vector from the reconstructed matrix
    reconstructed_user_vector = reconstructed_matrix[user_id - 2]  # User IDs start at 2, so subtract 2
    
    # Iterate through all artists to recommend
    for i, score in enumerate(reconstructed_user_vector):
        # Check if the artist has been interacted with (score > 0) and if the artist ID is valid
        if user_artist_matrix.iloc[user_id - 2, i] == 0:  # Ensure we only recommend non-interacted artists
            artist_id = i  # The index of the artist in the matrix
            if artist_id not in recommendations:
                recommendations[artist_id] = score
            else:
                recommendations[artist_id] += score
    
    # Sort recommendations by score (highest first)
    sorted_recommendations = sorted(recommendations.items(), key=lambda x: x[1], reverse=True)
    
    # Convert artist IDs to names using the artist_id_to_name mapping
    recommended_artists = [(artist_id, artist_id_to_name.get(artist_id, "Unknown"), score)
                           for artist_id, score in sorted_recommendations[:top_n]]
    
    return recommended_artists

In [15]:
# Example: Get top 5 SVD-based recommendations for user with userID=2
user_id = 2
svd_recommendations = get_svd_recommendations(user_id, user_artist_matrix, artist_id_to_name, top_n=5)

# Display SVD-based recommendations
print("\nTop SVD-Based Recommendations for User 2:")
for artist_id, artist_name, score in svd_recommendations:
    print(f"Artist ID: {artist_id}, Artist: {artist_name}, Similarity Score: {score:.2f}")


Top SVD-Based Recommendations for User 2:
Artist ID: 3464, Artist: Counting Crows, Similarity Score: 2346.16
Artist ID: 1089, Artist: Suede, Similarity Score: 1826.24
Artist ID: 259, Artist: 9th Wonder, Similarity Score: 1581.01
Artist ID: 153, Artist: De/Vision, Similarity Score: 1536.43
Artist ID: 992, Artist: Chris Rea, Similarity Score: 1110.52


In [16]:
# Example: Get top 5 SVD-based recommendations for user with userID=3
user_id = 3
svd_recommendations = get_svd_recommendations(user_id, user_artist_matrix, artist_id_to_name, top_n=5)

# Display SVD-based recommendations
print("\nTop SVD-Based Recommendations for User 3:")
for artist_id, artist_name, score in svd_recommendations:
    print(f"Artist ID: {artist_id}, Artist: {artist_name}, Similarity Score: {score:.2f}")


Top SVD-Based Recommendations for User 3:
Artist ID: 184, Artist: James Blunt, Similarity Score: 6.19
Artist ID: 148, Artist: The Boats, Similarity Score: 4.82
Artist ID: 1089, Artist: Suede, Similarity Score: 3.29
Artist ID: 151, Artist: Deep Forest, Similarity Score: 2.92
Artist ID: 298, Artist: Lily Allen, Similarity Score: 2.87


The recommendations for user 2 had significantly higher similarity scores compared to the user-based and item-based methods from before. However, for user 3, the SVD gave higher similarities than the user-based method but lower than the item-based method. This suggests that the performance of the method is highly dependenet on the available data and that this must be considered when choosing which technique we use.

#### ALS Using `implicit`
We now implement the very popular ALS method using the `implicit` library for efficiency. We use the csr_matrix() function from `scipy` to convert the user-artist matrix to a sparse format that is suitbale fort ALS.

In [17]:
def get_als_recommendations(user_id, user_artist_matrix, artist_id_to_name, top_n=5, factors=50, regularization=0.1, iterations=20):
    # Convert the user-artist matrix to sparse format (csr_matrix)
    sparse_matrix = csr_matrix(user_artist_matrix.values)
    
    # Initialize and train the ALS model
    model = implicit.als.AlternatingLeastSquares(factors=factors, regularization=regularization, iterations=iterations)
    model.fit(sparse_matrix)

    # Get the user's interaction vector (row from sparse matrix)
    user_vector = sparse_matrix[user_id]

    # Get top N artist recommendations (returns artist IDs and scores)
    recommendations = model.recommend(user_id, user_vector, N=top_n)

    # Convert artist IDs to artist names using the provided dictionary
    recommended_artists = [(artist_id_to_name[artist_id], score) for artist_id, score in zip(recommendations[0], recommendations[1])]

    return recommended_artists

In [18]:
# Example: Get top 5 ALS-based recommendations for user with userID=2
user_id = 2
als_recommendations = get_als_recommendations(user_id, user_artist_matrix, artist_id_to_name, top_n=5)

# Display ALS-based recommendations
print(f"\nTop ALS-Based Recommendations for User {user_id}:")
for artist_name, score in als_recommendations:
    print(f"Artist: {artist_name}, Predicted Listening Count: {score:.2f}")

c:\Users\milse\AppData\Local\Programs\Python\Python311\Lib\site-packages\implicit\cpu\als.py:95: RuntimeWarning: OpenBLAS is configured to use 8 threads. It is highly recommended to disable its internal threadpool by setting the environment variable 'OPENBLAS_NUM_THREADS=1' or by calling 'threadpoolctl.threadpool_limits(1, "blas")'. Having OpenBLAS use a threadpool can lead to severe performance issues here.
  check_blas_config()


  0%|          | 0/20 [00:00<?, ?it/s]


Top ALS-Based Recommendations for User 2:
Artist: The White Stripes, Predicted Listening Count: 1.28
Artist: Сплин, Predicted Listening Count: 1.22
Artist: Snow Patrol, Predicted Listening Count: 1.17
Artist: Ace of Base, Predicted Listening Count: 1.13
Artist: B.o.B, Predicted Listening Count: 1.11


In [19]:
# Example: Get top 5 ALS-based recommendations for user with userID=3
user_id = 8
als_recommendations = get_als_recommendations(user_id, user_artist_matrix, artist_id_to_name, top_n=5)

# Display ALS-based recommendations
print(f"\nTop ALS-Based Recommendations for User {user_id}:")
for artist_name, score in als_recommendations:
    print(f"Artist: {artist_name}, Predicted Listening Count: {score:.2f}")

  0%|          | 0/20 [00:00<?, ?it/s]


Top ALS-Based Recommendations for User 8:
Artist: Black Label Society, Predicted Listening Count: 1.41
Artist: Damien Rice, Predicted Listening Count: 1.33
Artist: Sia, Predicted Listening Count: 1.33
Artist: 2009 Names in Vain, Predicted Listening Count: 1.26
Artist: Starsailor, Predicted Listening Count: 1.25


We can see that this method returns predicted listening counts, unlike our other methods. All of the predicted listening counts for the top 5 recommendations are similar and are in the range 1.22-1.25. The artists are quite different in terms of genre.

## ALS User-based Collaborative Filtering with PySpark
We now implement ALS using PySpark methods to improve the efficiency of our recommender system, since it is highly scalable if we were to use our recommender system for very large datasets. We will try user-based and item-based implementations.

In [20]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F
from pyspark.ml.recommendation import ALS

# Start Spark session
spark = SparkSession.builder.appName("CollaborativeFilteringALS").getOrCreate()

# Convert cleaned pandas DataFrames to PySpark DataFrames
artists_spark_df = spark.createDataFrame(artists_cleaned)
user_artists_spark_df = spark.createDataFrame(user_artists_cleaned)

### User-based PySpark ALS

In [21]:
# ALS model setup for user-based collaborative filtering
# listening counts are implicit feedback, we are not starting from a cold-start
als = ALS(userCol="userID", itemCol="artistID", ratingCol="weight", coldStartStrategy="drop", implicitPrefs=True)

# Fit the ALS model
model = als.fit(user_artists_spark_df)

# Generate recommendations
user_recommendations = model.recommendForAllUsers(5)

In [22]:
# Create a dictionary to map artistID to artistName
artist_id_to_name = {row['id']: row['name'] for row in artists_spark_df.collect()}

# Function to map artistID to artistName and round scores to 2 decimal places
def map_recommendations(user_recommendations):
    def map_row(row):
        recommendations_with_names = [
            (artist_id_to_name.get(rec[0], "Unknown"), round(rec[1], 2)) for rec in row['recommendations']
        ]
        return (row['userID'], recommendations_with_names)

    mapped_recommendations = user_recommendations.rdd.map(map_row).toDF(["userID", "recommendations"])
    return mapped_recommendations

# Apply the artistID to name mapping function
user_recommendations_with_names = map_recommendations(user_recommendations)

# Show the final recommendations with artist names and rounded scores
user_recommendations_with_names.show(truncate=False)

+------+------------------------------------------------------------------------------------------------------------------------------------------------+
|userID|recommendations                                                                                                                                 |
+------+------------------------------------------------------------------------------------------------------------------------------------------------+
|3     |[{Ani DiFranco, 1.19}, {Billie Holiday, 1.17}, {Craig Armstrong, 1.13}, {Ellen Allien, 1.1}, {Auf der Maur, 1.09}]                              |
|5     |[{Joy Division, 1.1}, {PJ Harvey, 1.1}, {Jeff Buckley, 1.09}, {Blur, 1.08}, {Interpol, 1.08}]                                                   |
|6     |[{T.I., 0.51}, {Kanye West, 0.51}, {Kelis, 0.51}, {Timbaland, 0.5}, {Black Eyed Peas, 0.5}]                                                     |
|12    |[{Alexandre Desplat, 1.45}, {Guano Apes, 1.42}, {Tokio Hotel, 1.31},

We can see that the output is recommendations for all users and this was computed very quickly, so the use of PySpark is effective.

Also, the output gives recommendations with a 'score'. This is the relative confidence that a user will like a given artist. Clearly, some users, like user 16, have higher score but some users, like user 28, have much lower scores. This would indicate that the recommendations for user 16 relative to user 28 are much better. This is likely dependent on the availability of data for the different users.

We will analyse the distribution of the scores.

In [23]:
# Explode the recommendations column into individual rows
exploded_user_recommendations = user_recommendations.withColumn("recommendation", F.explode("recommendations"))

# Extract artistID and rating
exploded_user_recommendations = exploded_user_recommendations.select(
    F.col("userID"),
    F.col("recommendation.artistID").alias("artistID"),  # Extract artistID
    F.col("recommendation.rating").alias("score")        # Extract rating as score
)

# Generate summary statistics for the scores
summary_stats = exploded_user_recommendations.select("score").summary()

# Display the summary statistics
summary_stats.show()

+-------+------------------+
|summary|             score|
+-------+------------------+
|  count|              9460|
|   mean|1.1139602977502647|
| stddev|0.2649619654141234|
|    min|       3.74318E-40|
|    25%|         1.0361626|
|    50%|         1.1287955|
|    75%|         1.2545294|
|    max|         1.9816878|
+-------+------------------+



We can see that the scores range from around 0 to 2, with most of the recommendations being between 1.03 and 1.25. We then will say that the best recommendations are given by scores above 1.25.

### Item-based PySpark ALS

In [24]:
# ALS model setup for item-based collaborative filtering
# Swap userCol and itemCol for item-based filtering
# listening counts are implicit feedback, we are not starting from a cold-start
als = ALS(userCol="artistID", itemCol="userID", ratingCol="weight", coldStartStrategy="drop", implicitPrefs=True)

# Fit the ALS model
model = als.fit(user_artists_spark_df)

# Generate item-based recommendations for each artist (item)
item_recommendations = model.recommendForAllItems(5)

In [25]:
# Apply the artistID to name mapping function
item_recommendations_with_names = map_recommendations(item_recommendations)

# Show the final recommendations with artist names and rounded scores
item_recommendations_with_names.show(truncate=False)

+------+---------------------------------------------------------------------------------------------------------------------------------+
|userID|recommendations                                                                                                                  |
+------+---------------------------------------------------------------------------------------------------------------------------------+
|3     |[{Alina Orlova, 1.26}, {Shpongle, 1.23}, {Agua de Annique, 1.23}, {Howard Shore, 1.19}, {Vibrasphere, 1.18}]                     |
|5     |[{Beck, 1.09}, {Portishead, 1.08}, {Joy Division, 1.07}, {Radiohead, 1.07}, {The Smashing Pumpkins, 1.07}]                       |
|6     |[{Cyndi Lauper, 0.52}, {Eminem, 0.49}, {Whitney Houston, 0.49}, {Michael Jackson, 0.49}, {Mary J. Blige, 0.49}]                  |
|12    |[{The Runaways, 1.35}, {Anti-Flag, 1.33}, {The Distillers, 1.32}, {Stereophonics, 1.29}, {The 69 Eyes, 1.27}]                    |
|13    |[{Eminem, 0.93}, {L

In [26]:
# Explode the recommendations column into individual rows
exploded_item_recommendations = item_recommendations.withColumn("recommendation", F.explode("recommendations"))

# Extract artistID and rating from the struct fields
exploded_item_recommendations = exploded_item_recommendations.select(
    F.col("userID"),
    F.col("recommendation.artistID").alias("artistID"),  # Extract artistID
    F.col("recommendation.rating").alias("score")        # Extract rating as score
)

# Generate summary statistics for the scores
summary_stats = exploded_item_recommendations.select("score").summary()

# Display the summary statistics
summary_stats.show()

+-------+------------------+
|summary|             score|
+-------+------------------+
|  count|              9460|
|   mean|1.1126823240899466|
| stddev|0.2660025667307528|
|    min|     1.6230643E-38|
|    25%|         1.0343101|
|    50%|         1.1248131|
|    75%|         1.2521037|
|    max|         2.1123302|
+-------+------------------+



The scores for the item-based implementation have a similar distribution but the maximum score is higher, so some of the recommendations from the item-based model are much better than by the user-based, depending on the given user.

## Conclusion
We have investigated collaborative filtering through memory-based and model-based methods. All of these methods gave recommendations which ranged in quality and performance depending on the availability of data. The model-based methods were much more efficient and allowed quick recommendations to be given. Using PySpark to implement ALS also increased efficiency significantly and would allow us to apply our recommendation methods to large datasets. This would be vital in developing our music recommendation system which could theoretically have millions of users.

---

## **References**
[1] F.O. Isinkaye, Y.O. Folajimi, B.A. Ojokoh,
Recommendation systems: Principles, methods and evaluation,
Egyptian Informatics Journal,
Volume 16, Issue 3,
2015,
Pages 261-273.
(https://www.sciencedirect.com/science/article/pii/S1110866515000341)

[2] Implicit Documentation: https://benfred.github.io/implicit/

[3] PySpark Collaborative Filtering Documentation: https://spark.apache.org/docs/latest/ml-collaborative-filtering.html

[4] scikit-learn Documentation: https://scikit-learn.org/stable/modules/generated/sklearn.decomposition.TruncatedSVD.html

[5] csr_matrix Documentation: https://docs.scipy.org/doc/scipy/reference/generated/scipy.sparse.csr_matrix.html
 
[6] PySpark ALS Documentation: https://spark.apache.org/docs/latest/api/python/reference/api/pyspark.ml.recommendation.ALS.html